# Assignment #2: Language models

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [6]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
len(set(words))

44256

Lowercased text

In [8]:
words=tokenize(corpus.lower())
len(set(words))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
# Write your code
nonletter = r'[^\p{L}.;:?!]'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
# Write your code here
def clean(text):
    return re.sub(nonletter, r'\u0020', text)

In [11]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [12]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa. Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [13]:
# Write your code here
sentence_boundaries = r'[.;:?!][\u0020]+(\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [14]:
# Write your code here
sentence_markup = r'\u0020</s>\n<s>\u0020\1'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [15]:
# Write your code here
text= re.sub(sentence_boundaries, sentence_markup, test_para)

In [16]:
print(text)

En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.


The output should look like this:

`En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.`

Insert markup codes in the beginning and end of the text

In [17]:
# Write your code here
text = '<s> '+ text + ' </s>'
print(text)

<s> En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden. </s>


The output should look like this:

`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>`

Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [18]:
# Write your code here
text = re.sub('\u0020+','\u0020',text)
text = re.sub('\.','',text)
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


The output should look like this:
    
`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s>`

Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [19]:
# Write your code here
def segment_sentences(text):
    text = clean(text)
    text= re.sub(sentence_boundaries, sentence_markup, text)
    text = '<s> '+ text + ' </s>'
    text = re.sub(r'[.;:?!]','\u0020',text)
    text = re.sub('\u0020+','\u0020',text)
    return text.lower()
    


In [20]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [21]:
corpus = segment_sentences(corpus)# Write your code here

In [22]:
print(corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this:

```
<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> 
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>
```

You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [23]:
# Write your code here
words= corpus.split()

In [24]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


The five last lines of the corpus should like this:

`['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']`



### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [25]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [26]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [27]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [28]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]
print(len(frequency_bigrams.items()))
print(len(words)-1)

320129
1041578


In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_. 

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [29]:
def unigram_lm(frequency, sent_words): # Write your code
    total_words=len(words);
    prob_uni=1;
    print('=====================================================')
    print('wi      C(wi)      #words      P(wi)')
    print('=====================================================')
    for word in sent_words:
        print(word + ' '*6 + str(frequency[word]) + ' '*6 + str(total_words) + ' '*6 + str(frequency[word]/total_words))
        prob_uni*=frequency[word]/total_words;
    print('=====================================================')
    print('Prob. unigrams:     ' + str(prob_uni))
    print('Geometric mean prob.: ' + str(prob_uni**(1/len(sent_words))))
    entropy_rate = -1/len(sent_words)*math.log(prob_uni,2)
    print('Entropy rate:     ' + str(entropy_rate))
    perplexity=2**entropy_rate
    print('Perplexity:     ' + str(perplexity))
    return perplexity;
        

In [30]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [31]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi      C(wi)      #words      P(wi)
det      21108      1041579      0.020265385534846612
var      12090      1041579      0.011607376876837956
en      13514      1041579      0.012974531936607785
gång      1332      1041579      0.0012788276261330154
en      13514      1041579      0.012974531936607785
katt      16      1041579      1.5361292806402586e-05
som      16288      1041579      0.015637796076917832
hette      97      1041579      9.312783763881568e-05
nils      87      1041579      8.352702963481407e-05
</s>      59047      1041579      0.056689891021228345
Prob. unigrams:     5.364136934636431e-27
Geometric mean prob.: 0.0023602064104148853
Entropy rate:     8.726871249541004
Perplexity:     423.6917566138702


In [32]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [39]:
def bigram_lm(frequency, frequency_bigrams, sent_words): # Write your code
    #prob_bi=1;
    prob_bi=frequency_bigrams[('<s>',sent_words[0])]/frequency['<s>'];
    print('=====================================================')
    print('wi      wi+1      Ci,i+1      C(i)      P(wi+1|wi)')
    print('=====================================================')
    print('<s>' + ' '*6 + str(sent_words[0])+ ' '*6 +str(frequency_bigrams[('<s>',sent_words[0])]) + ' '*6 + str(frequency['<s>']) +  ' '*6 + str(frequency_bigrams[('<s>',sent_words[0])]/frequency['<s>']))
    for i in range(len(sent_words)-1):
        if (sent_words[i],sent_words[i+1]) in frequency_bigrams:
            print(str(sent_words[i])+ ' '*6 + str(sent_words[i+1])+ ' '*6 +str(frequency_bigrams[(sent_words[i],sent_words[i+1])]) + ' '*6 + str(frequency[sent_words[i]]) +  ' '*6 + str(frequency_bigrams[(sent_words[i],sent_words[i+1])]/frequency[sent_words[i]]))
            prob_bi*=frequency_bigrams[(sent_words[i],sent_words[i+1])]/frequency[sent_words[i]];
        else:
            print(str(sent_words[i])+ ' '*6 + str(sent_words[i+1])+ ' '*6 + '0' + ' '*6 + str(frequency[sent_words[i]]) +  ' '*6 + '0' + ' '*6 + '*backoff:' + ' '*6 + str(frequency[sent_words[i+1]]/len(words)))
            prob_bi*=frequency[sent_words[i+1]]/len(words);
    print('=====================================================')
    print('Prob. bigrams:     ' + str(prob_bi))
    print('Geometric mean prob.: ' + str(prob_bi**(1/len(sent_words))))
    entropy_rate = -1/len(sent_words)*math.log(prob_bi,2)
    print('Entropy rate:     ' + str(entropy_rate))
    perplexity=2**entropy_rate
    print('Perplexity:     ' + str(perplexity))
    return perplexity;

In [42]:
sentences = ['det var en gång en katt som hette nils </s>','borta bra men hemma bäst </s>', 'lika barn leka bäst </s>','man kan inte lära en gammal hund att sitta </s>', 'man ska inte ropa hej förrän man kommit över bäcken </s>', 'många bäckar små </s>']
for sentence in sentences:
    sent_words = sentence.split()
    bigram_lm(frequency, frequency_bigrams, sent_words)
    
#sent_words
#perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi      wi+1      Ci,i+1      C(i)      P(wi+1|wi)
<s>      det      5672      59047      0.09605907158704083
det      var      3839      21108      0.1818741709304529
var      en      712      12090      0.058891645988420185
en      gång      706      13514      0.052242119283705785
gång      en      20      1332      0.015015015015015015
en      katt      6      13514      0.0004439840165754033
katt      som      2      16      0.125
som      hette      45      16288      0.002762770137524558
hette      nils      0      97      0      *backoff:      8.352702963481407e-05
nils      </s>      2      87      0.022988505747126436
Prob. bigrams:     2.376126423796318e-19
Geometric mean prob.: 0.013727357824989852
Entropy rate:     6.186802220488124
Perplexity:     72.84723052673391
wi      wi+1      Ci,i+1      C(i)      P(wi+1|wi)
<s>      borta      3      59047      5.080698426677054e-05
borta      bra      0      464      0      *backoff:      0.0004473976529864753
bra      men      3

In [35]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program it following the model of bigrams.

In [36]:
def trigrams(words): # Write your code
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [37]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [38]:
starting_text = 'De'.lower()

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies.

In [39]:
cand_nbr = 5

In [40]:
candidates = sorted(frequency_bigrams.items(), key=lambda x:x[1], reverse=True)
candidates = [x for x in candidates if x[0][1][:2]==starting_text]
candidates = [x[0][1] for x in candidates if x[0][0]=='<s>']
current_word_predictions_1 = candidates[:5];
#print(current_word_predictions_1[:5])

In [41]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [42]:
current_text = "Det var en ".lower()
#print(current_text.split())


Tokenize this text and return a list of tokens. Call it `tokens`.

In [43]:
candidates = sorted(frequency_trigrams.items(), key=lambda x:x[1], reverse=True)# Write your code here
#print(candidates[:10])
current_words = current_text.split()
candidates = [x for x in candidates if x[0][0]==current_words[-2]];
candidates = [x for x in candidates if x[0][1]==current_words[-1]];
tokens=[x[0][2] for x in candidates];
print(tokens[:5])

['stor', 'liten', 'gammal', 'god', 'sådan']


In [44]:
tokens

['stor',
 'liten',
 'gammal',
 'god',
 'sådan',
 'av',
 'så',
 'ung',
 'riktig',
 'gång',
 'dag',
 'man',
 'annan',
 'lång',
 'smula',
 'vacker',
 'mycket',
 'bra',
 'som',
 'fattig',
 'lycka',
 'besynnerlig',
 'präktig',
 'hög',
 'af',
 'karl',
 'människa',
 'förfärlig',
 'skön',
 'helt',
 'svår',
 'avsatt',
 'mästare',
 'märkvärdig',
 'olycka',
 'mängd',
 'vanlig',
 'simpel',
 'mörk',
 'och',
 'märklig',
 'rätt',
 'svag',
 'ganska',
 'sträng',
 'allvarsam',
 'kväll',
 'kyrka',
 'orimligt',
 'örn',
 'söndag',
 'baddare',
 'stark',
 'rik',
 'duktig',
 'vän',
 'fiende',
 'myndig',
 'kista',
 'väg',
 'snäll',
 'utsänd',
 'ny',
 'stolt',
 'kvinna',
 'bild',
 'grann',
 'sann',
 'kristen',
 'julafton',
 'dråplig',
 'hälsning',
 'främmande',
 'glädje',
 'underlig',
 'flitig',
 'tid',
 'ruin',
 'på',
 'dröm',
 'burgen',
 'lek',
 'dotter',
 'storrövare',
 'kringbyggd',
 'milstolpe',
 'hederssak',
 'skara',
 'fullt',
 'drummel',
 'landstrykare',
 'djärv',
 'vådlig',
 'stad',
 'grov',
 'fågel',


Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`

In [45]:
next_word_predictions = tokens[:5] # Write your code here

In [46]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [47]:
current_text = "Det var en g".lower()

In [48]:
candidates = sorted(frequency_trigrams.items(), key=lambda x:x[1], reverse=True)# Write your code here
#print(candidates[:10])
current_words = current_text.split()
candidates = [x for x in candidates if x[0][0]==current_words[-3]];
candidates = [x for x in candidates if x[0][1]==current_words[-2]];
candidates = [x for x in candidates if x[0][2][0]==current_words[-1]];
tokens=[x[0][2] for x in candidates];
#print(tokens[:5])
current_word_predictions_2 = tokens[:5]

In [49]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'grann']

## Checked answers

The system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [50]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'grann'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [51]:
STIL_ID = ["ni5812jo-s"] # Write your stil ids as a list
#CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
#                                     "2-language_models_solution.ipynb") # Write the name of your notebook
CURRENT_NOTEBOOK_PATH='2-language_models.ipynb';

The submission code will send your answer. It consists of the perplexities and predictions.

In [52]:
ANSWER = str((perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2))
ANSWER

"(423, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'grann'])"

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [53]:
#SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"
SUBMISSION_NOTEBOOK_PATH = "2-language_models.ipynb.submission.bz2"

In [54]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [55]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '5b585bd251946655527ac9940aa89a08391bb579a879d965c0dae6af0aec30649c3f73351184f8ceb3331a1c359d5004e561921cff087022f73edda94bbdbc42',
 'submission_id': 'fb8cae54-0a4f-4fb9-9ccd-685a252da279'}

## Reading

<p>As an application of n-grams, execute the Jupyter notebook by Peter Norvig <a
        href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">
    here</a>. Just run all the cells and be sure that you understand the code.
    You will find the data <a href="http://norvig.com/ngrams/">here</a>.</p>
<p>In your report, you will also describe one experiment with a long string of words
    your will create yourself or copy from a text you like. You will remove all the punctuation and
    white spaces from this string. Set this string in lowercase letters.</p>
<p>You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and
    run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. </p>
<p>You will comment the segmentation results you obtain with unigram and bigram models.
</p>